In [11]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from tqdm import tqdm

In [10]:
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [18]:
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
%%capture
!pip install contextualized-topic-models==2.2.0

In [78]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
stop_words.update(('\r', '\n', '\r\n', '\n\r'))

In [79]:
df = pd.read_csv("/content/drive/MyDrive/metacritic_user_reviews.csv")
df

,Unnamed: 0,title,review
0,0,squid_game,"Squid Game is average show. Average actors, av..."
1,1,squid_game,If you’ve seen Asian game shows or TV challeng...
2,2,squid_game,"Its not bad, Just over-rated. I think that peo..."
3,3,squid_game,Squid Game is of the Puzzle-Adventure-Mystery-...
4,4,squid_game,One of the biggest sensations on Netflix right...
...,...,...,...
101,101,k_zombie,Some of the contrivances in this are just insa...
102,102,k_zombie,"The show is close to a 10, highly addictive an..."
103,103,k_zombie,"Entertaining for the first half, high intensit..."
104,104,k_zombie,Very entertaining thrilling\r\nSome minor flaw...


In [80]:
sentence = df['review'].to_list()
sentence

['Squid Game is average show. Average actors, average characters etc. Only reasons why im not voting for 4 is that there are some well made plots and its an original show. Squid game did great in creating specific games, music and characters relationships. Now the negatives. Squid Game did terribly in doing the layout of story. There are many unrealistic moments just to keep the important characters alive. Story can be chaotic and you can get easily lost. Ending of 1st season is terrible and unrealistic. You will just look at it and wonder how could someone come up with this nonsense. Squid Game is little bit overhyped show. Its not bad and I really enjoyed many parts but that storywriting, oh boy.',
 'If you’ve seen Asian game shows or TV challenge competitions; you know how wacky they can be. This Korean series goes a step further. A group of people who have extreme debts are whisked away to an undisclosed location for a series of trials and competitions. If they win, they get big bu

In [81]:
sent = ",".join(sentence)
sent

'Squid Game is average show. Average actors, average characters etc. Only reasons why im not voting for 4 is that there are some well made plots and its an original show. Squid game did great in creating specific games, music and characters relationships. Now the negatives. Squid Game did terribly in doing the layout of story. There are many unrealistic moments just to keep the important characters alive. Story can be chaotic and you can get easily lost. Ending of 1st season is terrible and unrealistic. You will just look at it and wonder how could someone come up with this nonsense. Squid Game is little bit overhyped show. Its not bad and I really enjoyed many parts but that storywriting, oh boy.,If you’ve seen Asian game shows or TV challenge competitions; you know how wacky they can be. This Korean series goes a step further. A group of people who have extreme debts are whisked away to an undisclosed location for a series of trials and competitions. If they win, they get big bucks, 

In [82]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [83]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        # pos_words = [word if word[1].startswith('V') else word if word[1].startswith('N') else word if word[1].startswith('J') else word if word[1].startswith('R') else word for word in tag_words]
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results



<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<input>:12: DeprecationWarning: invalid escape sequence \?
<ipython-input-83-003b93e0b3e3>:12: DeprecationWarning: invalid escape sequence \?
  text = re.sub('[-=+,#/\?:^$.@*

In [84]:
custom_tokenizer = CustomTokenizer(text_to_word_sequence)
custom_tokenizer(sentence)

['squid',
 'game',
 'average',
 'show',
 'average',
 'actor',
 'average',
 'character',
 'etc',
 'reason',
 'vote',
 'well',
 'make',
 'plot',
 'original',
 'show',
 'squid',
 'game',
 'great',
 'create',
 'specific',
 'game',
 'music',
 'character',
 'relationship',
 'negative',
 'squid',
 'game',
 'terribly',
 'layout',
 'story',
 'many',
 'unrealistic',
 'moment',
 'keep',
 'important',
 'character',
 'alive',
 'story',
 'chaotic',
 'get',
 'easily',
 'lose',
 'end',
 'season',
 'terrible',
 'unrealistic',
 'look',
 'wonder',
 'someone',
 'come',
 'nonsense',
 'squid',
 'game',
 'little',
 'bit',
 'overhyped',
 'show',
 'bad',
 'really',
 'enjoy',
 'many',
 'part',
 'storywriting',
 'boy',
 'youve',
 'see',
 'asian',
 'game',
 'show',
 'challenge',
 'competition',
 'know',
 'wacky',
 'korean',
 'series',
 'step',
 'far',
 'group',
 'people',
 'extreme',
 'debt',
 'whisk',
 'away',
 'undisclosed',
 'location',
 'series',
 'trial',
 'competition',
 'win',
 'get',
 'big',
 'buck',
 'lo

In [85]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [86]:
train_bow_embeddings = vectorizer.fit_transform(sentence)

In [87]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [88]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [89]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [90]:
train_contextualized_embeddings = bert_embeddings_from_list(sentence, "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [91]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [92]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=5, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [2120/2120]	Train Loss: 366.90616708431605	Time: 0:00:00.283064: : 20it [00:05,  3.40it/s]


In [93]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['zombie',
              'situation',
              'les',
              'love',
              'act',
              'series',
              'episode',
              'walk',
              'actually',
              'almost'],
             1: ['story',
              'twist',
              'look',
              'way',
              'clear',
              'also',
              'job',
              'contrive',
              'believable',
              'reason'],
             2: ['allant',
              'bravo',
              'autres',
              'baseball',
              'entertaining',
              'issue',
              'cou',
              'amuse',
              'humanistic',
              'screenplay'],
             3: ['unravel',
              'donner',
              'motivate',
              'suspanse',
              'attendons',
              'gonna',
              'finally',
              'best',
              'entraables',
              'version

In [94]:
%%capture
!pip install pyldavis

In [95]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [00:04,  2.02it/s]
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
